In [38]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import math

## Hyperparameters

In [39]:
epochs = 100
learning_rate = 0.0004
batch_size = 64


device = torch.device("mps")

## Load the Datasets and setup the Data Loaders (with data augmentation)

In [40]:
transform_normal = transforms.Compose([
    transforms.Resize((250, 250)),
    transforms.ToTensor()
])

transform_train = transforms.Compose([
    transforms.Resize((250, 250)),
    # Randomly change the brightness of the image
    transforms.ColorJitter(brightness=0.5),
    # Randomly flip the image horizontally
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),  # Randomly flip the image vertically
    transforms.RandomRotation(55),  # Randomly rotate the image by 45 degrees
    transforms.ToTensor(),
    # transforms.Normalize(mean = [0.0, 0.0, 0.0], std = [1.0, 1.0, 1.0]) # Takes each value for the channel, subtracts the mean and divides by the standard deviation (value - mean) / std
])

# Define the transformations
transformations1 = transforms.Compose(
    [transforms.ToTensor(), transforms.Resize((250, 250))])

# Load the dataset
training_dataset = torchvision.datasets.Flowers102(root='./data', split="train",
                                                   download=True, transform=transform_train)
testing_dataset = torchvision.datasets.Flowers102(root='./data', split="test",
                                                  download=True, transform=transformations1)
validation_dataset = torchvision.datasets.Flowers102(root='./data', split="val",
                                                     download=True, transform=transformations1)

# Create the dataloaders
train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testing_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

## Create The Model

In [41]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.flatten = nn.Flatten()
    self.relu = nn.PReLU()
    
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(16),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(32),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(64),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer4 = nn.Sequential(
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(128),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer5 = nn.Sequential(
        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(256),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer6 = nn.Sequential(
        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(256),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    
    # self.fc1 = nn.Linear(256 * 3 * 3, 1024)
    self.fc1 = nn.Linear(256 * 3 * 3, 102)
    self.drop = nn.Dropout(p=0.25)
    self.fc2 = nn.Linear(1024, 256)
    self.fc3 = nn.Linear(256, 102)  # Output layer for 102 classes

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.layer5(x)
    x = self.layer6(x)
    
    x = self.flatten(x)
    x = self.fc1(x)
    # x = self.relu(x)
    # x = self.drop(x)
    # x = self.fc2(x)
    # x = self.relu(x)
    # x = self.fc3(x)
    return x


In [42]:
device = torch.device("mps")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.3, total_iters=8)

## Network Accuracy Tests (Validation and Testing)

In [43]:
def NetworkAccuracyOnValidation():
    with torch.no_grad():
        num_class_correct = [0 for i in range(102)]
        num_class_samples = [0 for i in range(102)]
        total_correct = 0
        total_samples = 0
        for images, labels in validation_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            _, predictions = torch.max(outputs, 1)
            total_samples += labels.size(0)
            total_correct += (predictions == labels).sum().item()

            for i in range(len(labels)):
                label = labels[i]
                pred = predictions[i]
                if label == pred:
                    num_class_correct[label] += 1
                num_class_samples[label] += 1

        acc = 100.0 * total_correct / total_samples
        # print(f'Accuracy on validation set: {acc} %')
        return acc


def NetworkAccuracyOnTesting():
    with torch.no_grad():
        num_class_correct = [0 for i in range(102)]
        num_class_samples = [0 for i in range(102)]
        total_correct = 0
        total_samples = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            _, predictions = torch.max(outputs, 1)
            total_samples += labels.size(0)
            total_correct += (predictions == labels).sum().item()

            for i in range(len(labels)):
                label = labels[i]
                pred = predictions[i]
                if (label == pred):
                    num_class_correct[label] += 1
                num_class_samples[label] += 1

        acc = 100.0 * total_correct / total_samples
        print(f'Accuracy on testing set: {acc} %')

        for i in range(102):
            acc = 100.0 * num_class_correct[i] / num_class_samples[i]
            print(f'Accuracy of {i} : {acc} %')

        return acc

## Training and Testing loop

In [44]:
best_accuracy_epoch = 0
best_accuracy = 0

for epoch in range(epochs):  # Loop over the dataset multiple times
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader, 0):
        images = images.to(device)
        labels = labels.to(device)
        
        label_pred = model(images)
        loss = criterion(label_pred, labels)
        
        predicted = torch.max(label_pred, 1)[1]
        batch_corr = (predicted == labels).sum()
        batch_acc = batch_corr.item() / len(images)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # print(f"Epoch Number {epoch}, Index = {i}/{len(train_loader)-1}, Loss = {loss.item()}")
        
    scheduler.step()
    current_accuracy = NetworkAccuracyOnValidation()
    print(f"Epoch Number {epoch}, Loss = {loss.item()}, Accuracy = {current_accuracy}")
    if (current_accuracy > best_accuracy):
        best_accuracy = current_accuracy
        best_accuracy_epoch = epoch
        
print(f"Best accuracy on validation split: {best_accuracy} at epoch {best_accuracy_epoch}")
model.eval()
NetworkAccuracyOnTesting()


Epoch Number 0, Loss = 4.204793930053711, Accuracy = 7.1568627450980395
Epoch Number 1, Loss = 3.6936440467834473, Accuracy = 12.058823529411764
Epoch Number 2, Loss = 3.5107157230377197, Accuracy = 16.176470588235293
Epoch Number 3, Loss = 3.059312343597412, Accuracy = 18.529411764705884
Epoch Number 4, Loss = 2.471863031387329, Accuracy = 20.980392156862745


KeyboardInterrupt: 